In [ ]:
# ============================================
# CONFIGURATION - KAGGLE PATHS
# ============================================
import os

IS_KAGGLE = os.path.exists('/kaggle')
print(f"Running on Kaggle: {IS_KAGGLE}")

if IS_KAGGLE:
    # Your dataset paths (based on your Kaggle upload structure)
    DATASET_PATH = '/kaggle/input/satellite-property-data'
    
    TRAIN_PATH = f'{DATASET_PATH}/train(1).xlsx'
    TEST_PATH = f'{DATASET_PATH}/test2.xlsx'
    
    # Images are nested: mapbox_images (1)/mapbox_images/
    IMAGE_DIR = f'{DATASET_PATH}/mapbox_images (1)/mapbox_images'
    
    OUTPUT_DIR = '/kaggle/working'
else:
    # Local paths
    TRAIN_PATH = 'data/train.xlsx'
    TEST_PATH = 'data/test.xlsx'
    IMAGE_DIR = '/Users/user/Downloads/mapbox_images'
    OUTPUT_DIR = 'outputs'

print(f"✅ Train: {TRAIN_PATH} - exists: {os.path.exists(TRAIN_PATH)}")
print(f"✅ Test: {TEST_PATH} - exists: {os.path.exists(TEST_PATH)}")
print(f"✅ Images: {IMAGE_DIR} - exists: {os.path.exists(IMAGE_DIR)}")

# Count images
if os.path.exists(IMAGE_DIR):
    n_images = len([f for f in os.listdir(IMAGE_DIR) if f.endswith('.png')])
    print(f"✅ Found {n_images} images")

In [ ]:
# Install dependencies (if needed on Kaggle)
# !pip install openpyxl xgboost -q

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# XGBoost
from xgboost import XGBRegressor

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

# Settings
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Configuration - IMPROVED
CONFIG = {
    'target_col': 'price',
    'lat_col': 'lat',
    'lon_col': 'long',
    'image_size': 224,
    'seed': 42,
    'test_size': 0.2,
    'batch_size': 32,
    'epochs': 15,           # Epochs for fine-tuning
    'lr': 5e-4,             # Lower LR for fine-tuning
    'use_log_target': True, # Train on log(price) - often helps!
}

np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])
print(f"✅ Using log-transformed target: {CONFIG['use_log_target']}")

## 1. Load Data

In [ ]:
# Load datasets
train_df = pd.read_excel(TRAIN_PATH, engine='openpyxl')
test_df = pd.read_excel(TEST_PATH, engine='openpyxl')

train_df.columns = [c.strip() for c in train_df.columns]
test_df.columns = [c.strip() for c in test_df.columns]

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nColumns: {list(train_df.columns)}")

# CRITICAL: Check what images actually exist
print("\n🔍 Diagnosing image naming...")
sample_images = sorted(os.listdir(IMAGE_DIR))[:10]
print(f"First 10 images: {sample_images}")

# Check if 'id' column exists and matches image names
if 'id' in train_df.columns:
    print(f"\nTrain ID range: {train_df['id'].min()} to {train_df['id'].max()}")
    # Check if first few IDs match image files
    for i in range(min(5, len(train_df))):
        row_id = train_df.iloc[i]['id']
        img_by_id = f'img_{int(row_id)}.png'
        img_by_idx = f'img_{i}.png'
        id_exists = os.path.exists(os.path.join(IMAGE_DIR, img_by_id))
        idx_exists = os.path.exists(os.path.join(IMAGE_DIR, img_by_idx))
        print(f"Row {i}: id={row_id} | img_by_id={img_by_id} exists={id_exists} | img_by_idx={img_by_idx} exists={idx_exists}")

In [ ]:
# Quick EDA
train_df.head()

In [ ]:
# Price distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(train_df['price'], bins=50, color='steelblue', edgecolor='white')
axes[0].set_xlabel('Price ($)')
axes[0].set_title('Price Distribution')
axes[0].axvline(train_df['price'].median(), color='red', linestyle='--', label=f'Median: ${train_df["price"].median():,.0f}')
axes[0].legend()

axes[1].hist(np.log1p(train_df['price']), bins=50, color='seagreen', edgecolor='white')
axes[1].set_xlabel('Log(Price)')
axes[1].set_title('Log-Transformed Price')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/price_distribution.png', dpi=150)
plt.show()

## 2. Sample Satellite Images

In [ ]:
# IMPORTANT: Images are named by ROW INDEX (img_0.png to img_16165.png)
# NOT by the ID column (which has numbers like 9117000170)

USE_ID_FOR_IMAGES = False  # Images use row index, NOT ID column!

def get_image_path(idx):
    """Get image path by row index."""
    return os.path.join(IMAGE_DIR, f'img_{idx}.png')

print(f"✅ Images are named by ROW INDEX (img_0.png, img_1.png, ...)")
print(f"✅ Image range: img_0.png to img_16165.png")
print(f"➡️ USE_ID_FOR_IMAGES = {USE_ID_FOR_IMAGES}")

# Verify first few images exist
for i in range(3):
    exists = os.path.exists(get_image_path(i))
    print(f"   img_{i}.png exists: {exists}")

# Display sample images
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

sample_indices = np.random.choice(len(train_df), 9, replace=False)

for i, idx in enumerate(sample_indices):
    row = train_df.iloc[idx]
    img_path = get_image_path(idx)  # Use row index!
    
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[i].imshow(img)
    else:
        axes[i].text(0.5, 0.5, f'No Image\n{os.path.basename(img_path)}', ha='center', va='center')
        axes[i].set_facecolor('lightgray')
    
    axes[i].set_title(f'Price: ${row["price"]:,.0f}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('Sample Satellite Images with Prices', fontsize=14)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/sample_images.png', dpi=150)
plt.show()

## 3. Data Preprocessing

In [ ]:
# Feature columns (exclude non-features)
exclude_cols = {CONFIG['target_col'], 'date', 'id', CONFIG['lat_col'], CONFIG['lon_col']}
feature_cols = [c for c in train_df.columns if c not in exclude_cols and train_df[c].dtype in ['int64', 'float64']]
print(f"Feature columns ({len(feature_cols)}): {feature_cols}")

In [ ]:
# Train/validation split
train_data, val_data = train_test_split(train_df, test_size=CONFIG['test_size'], random_state=CONFIG['seed'])

# IMPORTANT: Use ORIGINAL ROW INDICES for image lookup (img_0.png, img_1.png, ...)
# The DataFrame index preserves original row numbers even after split
train_image_ids = train_data.index.tolist()  # Original row indices
val_image_ids = val_data.index.tolist()      # Original row indices

print(f"Training: {len(train_data)}, Validation: {len(val_data)}")
print(f"Sample train image indices: {train_image_ids[:5]}")
print(f"✅ Using row INDEX for image lookup (img_0.png, img_1.png, ...)")

In [ ]:
# Preprocessing
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

X_train = preprocessor.fit_transform(train_data[feature_cols])
X_val = preprocessor.transform(val_data[feature_cols])
X_test = preprocessor.transform(test_df[feature_cols])

# Use log-transformed target for better training
if CONFIG['use_log_target']:
    y_train = np.log1p(train_data[CONFIG['target_col']].values)
    y_val = np.log1p(val_data[CONFIG['target_col']].values)
    y_train_original = train_data[CONFIG['target_col']].values
    y_val_original = val_data[CONFIG['target_col']].values
    print("✅ Using log1p(price) as target - will transform back for evaluation")
else:
    y_train = train_data[CONFIG['target_col']].values
    y_val = val_data[CONFIG['target_col']].values
    y_train_original = y_train
    y_val_original = y_val

print(f"X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")
print(f"Target range: {y_train.min():.2f} to {y_train.max():.2f}")

## 4. Model 1: XGBoost (Tabular Only) - Baseline

In [ ]:
# Train XGBoost
xgb_model = XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=CONFIG['seed'],
    early_stopping_rounds=50,
    eval_metric='rmse',
    tree_method='hist',  # Faster on Kaggle
    device='cuda' if DEVICE == 'cuda' else 'cpu'
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100
)

In [ ]:
# Evaluation function
def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n📊 {model_name}:")
    print(f"   RMSE:  ${rmse:,.2f}")
    print(f"   R²:    {r2:.4f}")
    
    return {'model': model_name, 'RMSE': rmse, 'R2': r2}

# XGBoost predictions (on log scale if use_log_target=True)
xgb_pred_raw = xgb_model.predict(X_val)

# Convert back from log scale for evaluation
if CONFIG['use_log_target']:
    xgb_pred = np.expm1(xgb_pred_raw)  # Convert log(price) back to price
    xgb_results = evaluate_model(y_val_original, xgb_pred, 'XGBoost (Tabular Only)')
else:
    xgb_pred = xgb_pred_raw
    xgb_results = evaluate_model(y_val, xgb_pred, 'XGBoost (Tabular Only)')

## 5. Model 2: Hybrid Model (Tabular + Satellite Images)

In [ ]:
# Dataset class - IMPROVED with better augmentation
class PropertyDataset(Dataset):
    def __init__(self, X_tabular, y=None, image_ids=None, train_mode=True):
        self.X_tab = X_tabular.astype(np.float32)
        self.y = y.astype(np.float32) if y is not None else None
        self.image_ids = image_ids
        
        if train_mode:
            # More aggressive augmentation for training
            self.transform = T.Compose([
                T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
                T.RandomHorizontalFlip(0.5),
                T.RandomVerticalFlip(0.5),  # Satellite images can be flipped vertically
                T.RandomRotation(15),        # Small rotation
                T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transform = T.Compose([
                T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        
        self.blank = Image.new('RGB', (CONFIG['image_size'], CONFIG['image_size']), (128, 128, 128))
    
    def __len__(self):
        return len(self.X_tab)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx] if self.image_ids is not None else idx
        img_path = os.path.join(IMAGE_DIR, f'img_{int(img_id)}.png')
        
        if os.path.exists(img_path):
            try:
                img = Image.open(img_path).convert('RGB')
            except:
                img = self.blank
        else:
            img = self.blank
        
        img_tensor = self.transform(img)
        tab_tensor = torch.from_numpy(self.X_tab[idx])
        
        if self.y is None:
            return img_tensor, tab_tensor
        return img_tensor, tab_tensor, torch.tensor(self.y[idx])

print("✅ Dataset with improved augmentation: RandomVerticalFlip, Rotation, ColorJitter")

In [ ]:
# Hybrid Model - STABLE version with FROZEN CNN
class HybridModel(nn.Module):
    def __init__(self, tabular_dim):
        super().__init__()
        
        # CNN (ResNet18) - FREEZE ALL for stability
        backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        cnn_features = backbone.fc.in_features  # 512
        backbone.fc = nn.Identity()
        self.cnn = backbone
        
        # FREEZE entire CNN
        for param in self.cnn.parameters():
            param.requires_grad = False
        
        # CNN feature processor (trainable)
        self.cnn_processor = nn.Sequential(
            nn.Linear(cnn_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
        )
        
        # Tabular MLP - deeper network
        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
        )
        
        # Simple concatenation fusion (more stable than attention)
        self.head = nn.Sequential(
            nn.Linear(128, 64),  # 64 (img) + 64 (tab) = 128
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, img, tab):
        # Extract frozen CNN features
        with torch.no_grad():
            img_feat = self.cnn(img)
        
        # Process CNN features
        img_feat = self.cnn_processor(img_feat)
        
        # Tabular features
        tab_feat = self.tabular(tab)
        
        # Concatenate and predict
        combined = torch.cat([img_feat, tab_feat], dim=1)
        return self.head(combined).squeeze(1)

print("✅ STABLE Model: Frozen CNN + Concatenation Fusion")

In [ ]:
# Create data loaders - USE ORIGINAL PRICES (not log) for Hybrid model stability
# XGBoost uses log target, but Hybrid uses original prices

# For hybrid model, use original (non-log) targets
y_train_hybrid = y_train_original.astype(np.float32)
y_val_hybrid = y_val_original.astype(np.float32)

train_dataset = PropertyDataset(X_train, y_train_hybrid, image_ids=train_image_ids, train_mode=True)
val_dataset = PropertyDataset(X_val, y_val_hybrid, image_ids=val_image_ids, train_mode=False)

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")
print(f"✅ Hybrid model uses ORIGINAL prices (not log-transformed) for stability")
print(f"   Target range: ${y_train_hybrid.min():,.0f} to ${y_train_hybrid.max():,.0f}")

In [ ]:
# Initialize model - STABLE configuration
model = HybridModel(tabular_dim=X_train.shape[1]).to(DEVICE)

# Count trainable vs total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} ({100*trainable_params/total_params:.1f}%)")

# Simple optimizer - no differential LR needed since CNN is frozen
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3,
    weight_decay=1e-4
)

criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

print(f"✅ Learning rate: 1e-3, Scheduler: ReduceLROnPlateau")

In [ ]:
# Training loop - STABLE version (no log transform for hybrid)
best_rmse = float('inf')
best_state = None
history = {'train_loss': [], 'val_rmse': []}
patience_counter = 0
patience = 7

for epoch in range(1, CONFIG['epochs'] + 1):
    # Train
    model.train()
    train_loss = 0
    for img, tab, y in tqdm(train_loader, desc=f'Epoch {epoch}', leave=False):
        img, tab, y = img.to(DEVICE), tab.to(DEVICE), y.to(DEVICE)
        
        optimizer.zero_grad()
        pred = model(img, tab)
        loss = criterion(pred, y)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        train_loss += loss.item() * len(y)
    
    train_loss /= len(train_loader.dataset)
    
    # Validate
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for img, tab, y in val_loader:
            img, tab = img.to(DEVICE), tab.to(DEVICE)
            pred = model(img, tab)
            preds.extend(pred.cpu().numpy())
            targets.extend(y.numpy())
    
    # Direct RMSE (no log transform for hybrid)
    val_rmse = np.sqrt(mean_squared_error(targets, preds))
    
    history['train_loss'].append(train_loss)
    history['val_rmse'].append(val_rmse)
    
    scheduler.step(val_rmse)
    current_lr = optimizer.param_groups[0]['lr']
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        patience_counter = 0
        print(f"Epoch {epoch:02d} | Loss: {train_loss/1e9:.2f}B | Val RMSE: ${val_rmse:,.0f} | LR: {current_lr:.2e} ✓")
    else:
        patience_counter += 1
        print(f"Epoch {epoch:02d} | Loss: {train_loss/1e9:.2f}B | Val RMSE: ${val_rmse:,.0f} | LR: {current_lr:.2e}")
    
    if patience_counter >= patience:
        print(f"\n⏹️ Early stopping at epoch {epoch}")
        break

print(f"\n🏆 Best Val RMSE: ${best_rmse:,.0f}")

In [ ]:
# Load best model and evaluate
model.load_state_dict(best_state)
model.eval()

preds = []
with torch.no_grad():
    for img, tab, _ in val_loader:
        img, tab = img.to(DEVICE), tab.to(DEVICE)
        pred = model(img, tab)
        preds.extend(pred.cpu().numpy())

# Direct evaluation (no log transform for hybrid)
hybrid_preds = np.array(preds)
hybrid_results = evaluate_model(y_val_original, hybrid_preds, 'Hybrid (Tabular + Satellite)')

## 6. Model Comparison

In [ ]:
# Compare models
results_df = pd.DataFrame([xgb_results, hybrid_results])

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
display(results_df.round(2))

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

colors = ['steelblue', 'coral']

axes[0].bar(results_df['model'], results_df['RMSE'], color=colors)
axes[0].set_ylabel('RMSE ($)')
axes[0].set_title('RMSE (Lower is Better)')
axes[0].tick_params(axis='x', rotation=15)

axes[1].bar(results_df['model'], results_df['R2'], color=colors)
axes[1].set_ylabel('R² Score')
axes[1].set_title('R² (Higher is Better)')
axes[1].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/model_comparison.png', dpi=150)
plt.show()

# Improvements
xgb_rmse = xgb_results['RMSE']
hybrid_rmse = hybrid_results['RMSE']

print(f"\n📈 Results Summary:")
print(f"   XGBoost RMSE:  ${xgb_rmse:,.0f}")
print(f"   Hybrid RMSE:   ${hybrid_rmse:,.0f} ({100*(xgb_rmse-hybrid_rmse)/xgb_rmse:+.1f}% vs XGB)")

## 7. Generate Predictions

In [ ]:
# Create test dataset and generate XGBoost predictions
test_image_ids = list(range(len(test_df)))
print(f"Test samples: {len(test_df)}")

# Generate XGBoost predictions (convert from log scale)
xgb_test_preds_raw = xgb_model.predict(X_test)
if CONFIG['use_log_target']:
    test_preds = np.expm1(xgb_test_preds_raw)  # Convert log(price) back to price
else:
    test_preds = xgb_test_preds_raw

print(f"\n✅ Generated {len(test_preds)} XGBoost model predictions")

In [ ]:
# Create submission file
submission = pd.DataFrame({
    'id': test_df['id'] if 'id' in test_df.columns else range(len(test_preds)),
    'predicted_price': test_preds
})

submission.to_csv(f'{OUTPUT_DIR}/predictions.csv', index=False)
print(f"✅ Saved predictions to {OUTPUT_DIR}/predictions.csv")
submission.head()

In [ ]:
# Prediction statistics
print("\n📊 Prediction Statistics:")
print(f"   Mean:   ${test_preds.mean():,.2f}")
print(f"   Median: ${np.median(test_preds):,.2f}")
print(f"   Min:    ${test_preds.min():,.2f}")
print(f"   Max:    ${test_preds.max():,.2f}")

## 8. Grad-CAM Explainability

In [ ]:
import cv2

class GradCAMForFrozenCNN:
    """
    Grad-CAM implementation that works with frozen CNN backbones.
    Temporarily enables gradients for visualization only.
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.gradients = None
        self.activations = None
        self.handles = []
        
        # Register hooks
        self.handles.append(target_layer.register_forward_hook(self._forward_hook))
        self.handles.append(target_layer.register_full_backward_hook(self._backward_hook))
    
    def _forward_hook(self, module, input, output):
        # Store activations WITH gradients enabled
        self.activations = output
    
    def _backward_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]
    
    def generate(self, img, tab):
        """Generate Grad-CAM heatmap by temporarily enabling gradients."""
        self.model.eval()
        
        # Temporarily enable gradients for all CNN parameters
        original_requires_grad = {}
        for name, param in self.model.cnn.named_parameters():
            original_requires_grad[name] = param.requires_grad
            param.requires_grad = True
        
        try:
            # Need to override the forward to not use torch.no_grad()
            # Extract CNN features WITH gradients
            img_feat = self.model.cnn(img)
            img_feat_processed = self.model.cnn_processor(img_feat)
            tab_feat = self.model.tabular(tab)
            combined = torch.cat([img_feat_processed, tab_feat], dim=1)
            output = self.model.head(combined).squeeze(1)
            
            self.model.zero_grad()
            output.backward(retain_graph=True)
            
            if self.gradients is None:
                return None
            
            # Compute Grad-CAM
            weights = self.gradients.mean(dim=(2, 3), keepdim=True)
            cam = (weights * self.activations).sum(dim=1, keepdim=True)
            cam = torch.relu(cam).squeeze().cpu().detach().numpy()
            
            # Normalize
            if cam.max() - cam.min() > 1e-8:
                cam = (cam - cam.min()) / (cam.max() - cam.min())
            else:
                cam = np.zeros_like(cam)
            
            return cam
            
        finally:
            # Restore original requires_grad state
            for name, param in self.model.cnn.named_parameters():
                param.requires_grad = original_requires_grad[name]
    
    def remove_hooks(self):
        for handle in self.handles:
            handle.remove()

print("✅ GradCAM class ready (works with frozen CNN)")

In [ ]:
# Visualize Grad-CAM for sample properties
def visualize_gradcam_samples(model, val_loader, val_data, n_samples=3):
    """Generate Grad-CAM visualizations for sample properties."""
    
    # Get target layer (last conv layer of ResNet18)
    target_layer = model.cnn.layer4[-1].conv2
    gradcam = GradCAMForFrozenCNN(model, target_layer)
    
    # Get sample indices from validation set
    sample_indices = np.random.choice(len(val_data), n_samples, replace=False)
    
    fig, axes = plt.subplots(n_samples, 3, figsize=(15, 5*n_samples))
    if n_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i, sample_idx in enumerate(sample_indices):
        # Get data from dataloader
        img_id = val_image_ids[sample_idx]
        img_path = os.path.join(IMAGE_DIR, f'img_{int(img_id)}.png')
        
        if not os.path.exists(img_path):
            print(f"Image not found: {img_path}")
            continue
        
        # Load original image
        original_img = Image.open(img_path).convert('RGB')
        original_img_resized = original_img.resize((CONFIG['image_size'], CONFIG['image_size']))
        original_np = np.array(original_img_resized)
        
        # Prepare tensors
        transform = T.Compose([
            T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        img_tensor = transform(original_img).unsqueeze(0).to(DEVICE)
        tab_tensor = torch.from_numpy(X_val[sample_idx:sample_idx+1].astype(np.float32)).to(DEVICE)
        
        # Generate Grad-CAM
        cam = gradcam.generate(img_tensor, tab_tensor)
        
        if cam is None:
            print(f"Could not generate CAM for sample {i}")
            continue
        
        # Resize CAM to image size
        cam_resized = cv2.resize(cam, (CONFIG['image_size'], CONFIG['image_size']))
        
        # Create heatmap
        heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        
        # Overlay
        overlay = np.uint8(0.4 * heatmap + 0.6 * original_np)
        
        # Get actual price
        actual_price = y_val_original[sample_idx]
        
        # Plot
        axes[i, 0].imshow(original_np)
        axes[i, 0].set_title(f'Satellite Image\nPrice: \${actual_price:,.0f}')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(cam_resized, cmap='jet')
        axes[i, 1].set_title('Grad-CAM Heatmap')
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(overlay)
        axes[i, 2].set_title('Overlay\n(Red = High Importance)')
        axes[i, 2].axis('off')
    
    gradcam.remove_hooks()
    
    plt.suptitle('Grad-CAM: What the Model Sees in Satellite Images', fontsize=14)
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/gradcam_visualization.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✅ Saved Grad-CAM visualization to {OUTPUT_DIR}/gradcam_visualization.png")

# Run visualization
print("🔍 Generating Grad-CAM visualizations...")
visualize_gradcam_samples(model, val_loader, val_data, n_samples=3)

---
## Summary

In [ ]:
# Auto-generate Summary
from IPython.display import display, Markdown

xgb_rmse = xgb_results['RMSE']
hybrid_rmse = hybrid_results['RMSE']

best_model = results_df.loc[results_df['RMSE'].idxmin(), 'model']
best_rmse = results_df['RMSE'].min()

hybrid_improvement = ((xgb_rmse - hybrid_rmse) / xgb_rmse) * 100

# Get top features from XGBoost
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)
top_features = feature_importance.head(5)['feature'].tolist()

summary_md = f"""
### 🏆 Model Comparison Results

| Model | RMSE | R² |
|-------|------|-----|
| XGBoost (Tabular Only) | \\${xgb_results['RMSE']:,.0f} | {xgb_results['R2']:.4f} |
| **Hybrid (Tabular + Satellite)** | **\\${hybrid_results['RMSE']:,.0f}** | **{hybrid_results['R2']:.4f}** |

### Key Findings:
1. **Best Model:** {best_model} (RMSE: \\${best_rmse:,.0f})
2. Hybrid model improvement vs XGBoost: **{hybrid_improvement:+.2f}%**
3. Top 5 tabular features: **{', '.join(top_features)}**

### Improvements Applied:
- ✅ Frozen CNN (ResNet18) + trainable fusion layers
- ✅ Log-transformed target for XGBoost
- ✅ Enhanced image augmentation (rotation, color jitter)

### Files Generated:
- `predictions.csv` - Hybrid model test predictions
- `model_comparison.png` - Visual comparison
"""

display(Markdown(summary_md))
print("\n✅ Notebook completed successfully!")